# Contents
- [Imports](#imports)
- [Deployment](#deploy)
- [Future Plans](#plans)

---
# Imports<a id=imports></a>

In [1]:
import argparse
import logging
import time
import pickle
import re
import cv2
import numpy as np
import pandas as pd

from sklearn.svm import SVC

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework

---
# Deployment<a id=deploy></a>
We take the model that has given us the best results, which would be SVC after feature engineering, and we include the code below that will allow our classifier to work on a webcam.

In [2]:
logger = logging.getLogger('TfPoseEstimator-WebCam')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

fps_time = 0

In [3]:
parser = argparse.ArgumentParser(description='tf-pose-estimation realtime webcam')
parser.add_argument('--camera', type=int, default=0)

parser.add_argument('--resize', type=str, default='0x0',
                    help='if provided, resize images before they are processed. default=0x0, Recommends : 432x368 or 656x368 or 1312x736 ')
parser.add_argument('--resize-out-ratio', type=float, default=4.0,
                    help='if provided, resize heatmaps before they are post-processed. default=1.0')

parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin / mobilenet_v2_large / mobilenet_v2_small')
parser.add_argument('--show-process', type=bool, default=False,
                    help='for debug purpose, if enabled, speed for inference is dropped.')

_StoreAction(option_strings=['--show-process'], dest='show_process', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, help='for debug purpose, if enabled, speed for inference is dropped.', metavar=None)

In [4]:
model='SVC'

In [16]:
arguments='--model=cmu --resize-out-ratio=4' #resize-out-ratio affects upsampling, tweak this to affect speed vs accuracy


In [5]:
mod=pickle.load(open(r'.//data/model2/'+model+'.sav', 'rb'))
args = parser.parse_args(arguments.split())

In [17]:
def camtest(args):
    
    fps_time = 0
    
    lis=[]
    for x in range(1,18):
        lis.append('x'+str(x))
        lis.append('y'+str(x))
    header=pd.DataFrame(columns=lis)
    
    xparts=['x1','x2','x3','x4','x5','x6','x7','x8','x11']
    yparts=['y1','y2','y3','y4','y5','y6','y7','y8','y11']
    drop=['x0', 'x10', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x9', 'y0', 'y10', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y9']
    
    logger.debug('cam read+')
    cam = cv2.VideoCapture(args.camera)
    ret_val, image = cam.read()
    logger.info('cam image=%dx%d' % (image.shape[1], image.shape[0]))

    while True:
        ret_val, image = cam.read()

        logger.debug('image process+')
        humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=args.resize_out_ratio)
        logger.debug('postprocess+')
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

        logger.debug('show+')
        cv2.putText(image,
                    "FPS: %f" % (1.0 / (time.time() - fps_time)),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)    

        move='none'

        try:
            part=re.findall('BodyPart:(.*?)-',str(humans[0]))
            xco=re.findall('\((.*?),',str(humans[0]))
            yco=re.findall(', (.*?)\)',str(humans[0]))
            xpart=['x'+c for c in part]
            ypart=['y'+c for c in part]
            test1=pd.DataFrame([[float(x) for x in xco+yco]],columns=xpart+ypart)
            data=pd.concat([header,test1],sort=True)

            data.drop(columns=drop,inplace=True) #create our new features based on our previous feature engineering
            data[xparts]=data[xparts].subtract(data['x1'],axis=0)
            data[yparts]=data[yparts].subtract(data['y1'],axis=0)
            data.fillna(0,inplace=True)
    
            data['shwidth']=np.hypot((data.x2-data.x5),(data.y2-data.y5))
            data['splength']=np.hypot((data.x1-((data.x8+data.x11)/2)),(data.y1-((data.y8+data.y11)/2)))
            data['lhnsh']=np.hypot((data.x7-data.x5),(data.y7-data.y5))*data.shwidth/data.splength
            data['rhnsh']=np.hypot((data.x4-data.x2),(data.y4-data.y2))*data.shwidth/data.splength
            data['lelsh']=np.hypot((data.x6-data.x5),(data.y6-data.y5))*data.shwidth/data.splength
            data['relsh']=np.hypot((data.x3-data.x2),(data.y3-data.y2))*data.shwidth/data.splength #end of feature creation
            
            predict=mod.predict(data.to_numpy())
            if predict[0]==0:
                move='none'
            elif predict[0]==1:
                move='guard'
            elif predict[0]==2:
                move='jab'
            elif predict[0]==3:
                move='cross'
            elif predict[0]==4:
                move='hook'
            elif predict[0]==5:
                move='uppercut'
            else:
                move='none'

        except:
            move='not detected'

        finally:
            cv2.putText(image,
                        'move: '+move,
                        (10, 30),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 0, 255), 2)
            cv2.imshow('tf-pose-estimation result', image)
            fps_time = time.time()

            if cv2.waitKey(1) == 27:
                break
            
    cv2.destroyAllWindows()
    logger.debug('finished+')
    return(data)

In [18]:
camtest(args)

[2019-08-02 04:57:09,271] [TfPoseEstimator-WebCam] [DEBUG] cam read+
I0802 04:57:09.271672  9972 <ipython-input-17-0ae690e26da5>:15] cam read+
[2019-08-02 04:57:14,767] [TfPoseEstimator-WebCam] [INFO] cam image=640x480
I0802 04:57:14.767975  9972 <ipython-input-17-0ae690e26da5>:18] cam image=640x480
[2019-08-02 04:57:14,799] [TfPoseEstimator-WebCam] [DEBUG] image process+
I0802 04:57:14.799891  9972 <ipython-input-17-0ae690e26da5>:23] image process+
[2019-08-02 04:57:14,969] [TfPoseEstimator-WebCam] [DEBUG] postprocess+
I0802 04:57:14.969460  9972 <ipython-input-17-0ae690e26da5>:25] postprocess+
[2019-08-02 04:57:14,972] [TfPoseEstimator-WebCam] [DEBUG] show+
I0802 04:57:14.972430  9972 <ipython-input-17-0ae690e26da5>:28] show+
[2019-08-02 04:57:15,015] [TfPoseEstimator-WebCam] [DEBUG] image process+
I0802 04:57:15.015316  9972 <ipython-input-17-0ae690e26da5>:23] image process+
[2019-08-02 04:57:15,156] [TfPoseEstimator-WebCam] [DEBUG] postprocess+
I0802 04:57:15.156936  9972 <ipython-

,x1,x11,x2,x3,x4,x5,x6,x7,x8,y1,...,y5,y6,y7,y8,shwidth,splength,lhnsh,rhnsh,lelsh,relsh
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,NaN,NaN,NaN,NaN


---
# Future Plans<a id=plans></a>
We have hit our first milestone of creating a 'deployable' model for basic boxing-classification use.<br/>
This is an acceptable milestone to hit for the purposes of this capstone project. Moving onward, we can aim to increase the performance of the model and also to hit other milestones to create an actual product for consumer/business use.<br/>

To increase the performance of our model, we can do the following:
1. Increase the amount and variation of our training data
2. Use more advanced neural network methods to create our predictors

Our future milestones to hit include:
1. Creating a multi-label classifier to pinpoint [common mistakes](#https://www.infighting.ca/blog/the-77-most-common-mistakes-in-boxing/) in boxing movements
2. Deploying our predictors on either mobile phones (aimed to consumers) or on a Jetson TX2 (aimed to businesses) for actual product creation
3. Create a boxing response predictor, trained on actual boxing matches, to create an AI for boxing training.
4. Deploy the AI in step 3 on Unity for VR games, or deploy it on a [robot](https://youtu.be/wNLuZNLBegw?t=137)